In [ ]:
import os
import pandas as pd
import glob

In [ ]:
print(os.getcwd())
os.chdir("./TrainOps Results")
print(os.getcwd())

listScenarios= glob.glob('*/')

for j in listScenarios:
    print(j)

print(len(listScenarios))

voltmeterAllList = []

for f in listScenarios:

    os.chdir("./" + f)

    print("Current Dir: " , os.getcwd())

    voltmeterFiles = [f for f in glob.glob("*Voltmeters*.txt")]
    
    print(len(voltmeterFiles))

    dfAllList = []

    for i in voltmeterFiles:

        df = pd.read_table(i, sep="\t", skiprows=6)

        maxVolt= max(df["Voltage (V)"])
        minVolt= min(df["Voltage (V)"])
        fileName= i
        scenario= f.replace("/","")

        d = dict(((k, eval(k)) for k in ("scenario", "fileName", 'maxVolt', 'minVolt')))

        VoltDF = pd.DataFrame([d])

        dfAllList.append(VoltDF)

    voltDfAll = pd.concat(dfAllList, axis=0, ignore_index=True)

    path_parent = os.path.dirname(os.getcwd())

    os.chdir(path_parent)

    #voltmeterOutput = scenario + "_voltmeter_scenario.csv"

    #voltDfAll.to_csv(voltmeterOutput)

    voltmeterAllList.append(voltDfAll)

df_final = pd.concat(voltmeterAllList, axis=0, ignore_index=True)

In [ ]:
##df_final.reset_index(level=0, inplace=True)

In [ ]:
voltmeter_grp = df_final.groupby(['fileName'])

In [ ]:
max_volt_df = voltmeter_grp['maxVolt'].max().to_frame()
max_volt_df.to_csv("max_volt_df.csv")

min_volt_df = voltmeter_grp['minVolt'].min().to_frame()
min_volt_df.to_csv("min_volt_df.csv")

In [ ]:
max_df = pd.merge(max_volt_df, df_final, 
                  on=['maxVolt', 'fileName'] , how='left').drop(['minVolt'], axis=1)

In [ ]:
min_df = pd.merge(min_volt_df, df_final, 
                  on=['minVolt','fileName'] , how='left').drop(['maxVolt'], axis=1)

In [ ]:
final_df = pd.merge(max_df, min_df, on=['fileName'], 
         how='inner', suffixes=('_maxVolt', '_minVolt'))

In [ ]:
final_df.rename(columns={'fileName':'voltmeter'}, inplace=True)

In [ ]:
column_titles = ['voltmeter','maxVolt','scenario_maxVolt', 'minVolt', 'scenario_minVolt']

df = final_df.reindex(columns=column_titles)

In [ ]:
df.to_csv("voltmeter_summary.csv", index=False)